In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('Desktop/All GitHub Repo Projects/Bayesian Decision Theory - Fraud Detection/data/fraudTrain.csv')
t_df = pd.read_csv('Desktop/All GitHub Repo Projects/Bayesian Decision Theory - Fraud Detection/data/fraudTest.csv')
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [3]:
df['is_fraud'].value_counts()


is_fraud
0    1289169
1       7506
Name: count, dtype: int64

In [4]:
1289169+7506

1296675

In [5]:
print(df.columns)

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')


In [6]:
df = df.drop(['cc_num', 'trans_date_trans_time', 'first', 'last', 'dob', 'street', 'trans_num', 'unix_time', 'merchant','Unnamed: 0','city', 'zip'],axis=1, errors='ignore')
df

,category,amt,gender,state,lat,long,city_pop,job,merch_lat,merch_long,is_fraud
0,misc_net,4.97,F,NC,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,0
1,grocery_pos,107.23,F,WA,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,0
2,entertainment,220.11,M,ID,42.1808,-112.2620,4154,Nature conservation officer,43.150704,-112.154481,0
3,gas_transport,45.00,M,MT,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,0
4,misc_pos,41.96,M,VA,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...
1296670,entertainment,15.56,M,UT,37.7175,-112.4777,258,Geoscientist,36.841266,-111.690765,0
1296671,food_dining,51.70,M,MD,39.2667,-77.5101,100,"Production assistant, television",38.906881,-78.246528,0
1296672,food_dining,105.93,M,NM,32.9396,-105.8189,899,Naval architect,33.619513,-105.130529,0
1296673,food_dining,74.90,M,SD,43.3526,-102.5411,1126,Volunteer coordinator,42.788940,-103.241160,0


In [7]:
df.isnull().sum()

category      0
amt           0
gender        0
state         0
lat           0
long          0
city_pop      0
job           0
merch_lat     0
merch_long    0
is_fraud      0
dtype: int64

In [8]:
t_df = t_df.drop(['cc_num', 'trans_date_trans_time', 'first', 'last', 'dob', 'street', 'trans_num', 'unix_time', 'merchant','Unnamed: 0','city', 'zip'],axis=1, errors='ignore')
t_df

,category,amt,gender,state,lat,long,city_pop,job,merch_lat,merch_long,is_fraud
0,personal_care,2.86,M,SC,33.9659,-80.9355,333497,Mechanical engineer,33.986391,-81.200714,0
1,personal_care,29.84,F,UT,40.3207,-110.4360,302,"Sales professional, IT",39.450498,-109.960431,0
2,health_fitness,41.28,F,NY,40.6729,-73.5365,34496,"Librarian, public",40.495810,-74.196111,0
3,misc_pos,60.05,M,FL,28.5697,-80.8191,54767,Set designer,28.812398,-80.883061,0
4,travel,3.19,M,MI,44.2529,-85.0170,1126,Furniture designer,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...
555714,health_fitness,43.77,M,MO,40.4931,-91.8912,519,Town planner,39.946837,-91.333331,0
555715,kids_pets,111.84,M,TX,29.0393,-95.4401,28739,Futures trader,29.661049,-96.186633,0
555716,kids_pets,86.88,F,WA,46.1966,-118.9017,3684,Musician,46.658340,-119.715054,0
555717,travel,7.99,M,ID,44.6255,-116.4493,129,Cartographer,44.470525,-117.080888,0


In [9]:
t_df.isnull().sum()

category      0
amt           0
gender        0
state         0
lat           0
long          0
city_pop      0
job           0
merch_lat     0
merch_long    0
is_fraud      0
dtype: int64

In [10]:
from sklearn.preprocessing import OrdinalEncoder

# Define categorical columns
categorical_columns = ['category', 'job', 'state','gender']

# Initialize the encoder
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Fit on training data and transform both train & test data
df[categorical_columns] = oe.fit_transform(df[categorical_columns])
t_df[categorical_columns] = oe.transform(t_df[categorical_columns])  # Handles unseen values

# Store encoder for later use
label_encoders = oe

In [11]:
# Separate features and labels
X_train = df.drop(columns=['is_fraud'])
y_train = df['is_fraud']
X_test = t_df.drop(columns=['is_fraud'])
y_test = t_df['is_fraud']

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [14]:
fraud_mean = X_train[y_train == 1].mean(dim=0)
fraud_var = X_train[y_train == 1].var(dim=0) + 1e-6  # Avoid zero variance
legit_mean = X_train[y_train == 0].mean(dim=0)
legit_var = X_train[y_train == 0].var(dim=0) + 1e-6  # Avoid zero variance

In [15]:
def log_gaussian_pdf(x, mean, var):
    return -0.5 * (torch.log(2 * torch.pi * var) + ((x - mean) ** 2) / var)